In [1]:
import torch
import torch.nn as nn

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

    def forward(self, x):
        return self.embedding(x)

# Example
vocab_size = 10000  # Number of unique tokens
embed_size = 512    # Size of each embedding vector
embedding_layer = TokenEmbedding(vocab_size, embed_size)
sample_input = torch.randint(0, vocab_size, (2, 5))  # Batch of 2, sequence length 5
print(embedding_layer(sample_input))

tensor([[[-0.0555,  0.2981, -0.1093,  ..., -0.0584, -2.2460, -0.2288],
         [ 0.4222,  0.6497, -0.7676,  ..., -0.7496,  0.8840, -1.2974],
         [-0.8060,  1.6251, -1.7920,  ..., -0.4533, -0.4136,  0.0808],
         [ 0.6242,  2.0692, -0.0996,  ..., -2.2919,  0.0931, -1.5316],
         [-0.7041,  0.4715, -0.6178,  ..., -1.3172, -1.3331, -1.0782]],

        [[ 1.2602,  1.0408, -0.8412,  ...,  0.6926,  0.0530,  0.9972],
         [ 2.2176,  1.1191, -0.6826,  ...,  0.0735,  0.0847, -0.1087],
         [-0.5750,  1.0077,  0.1860,  ...,  1.0475, -1.2119, -0.6835],
         [ 0.0453,  1.3576, -1.3155,  ...,  0.2559, -1.0628, -1.3152],
         [ 0.1394,  2.3959,  0.6316,  ..., -1.9403,  0.3570,  0.9907]]],
       grad_fn=<EmbeddingBackward0>)


In [2]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_len=5000):
        super(PositionalEncoding, self).__init__()
        positional_encoding = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-math.log(10000.0) / embed_size))
        positional_encoding[:, 0::2] = torch.sin(position * div_term)
        positional_encoding[:, 1::2] = torch.cos(position * div_term)
        self.positional_encoding = positional_encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.positional_encoding[:, :x.size(1), :]

# Example
pos_encoding = PositionalEncoding(embed_size)
sample_embed = torch.randn(2, 5, embed_size)  # Batch of 2, sequence length 5
print(pos_encoding(sample_embed))

tensor([[[ 0.1125,  2.3726, -0.6392,  ...,  1.8683, -1.3322,  1.0999],
         [ 2.2893, -1.0485,  0.8637,  ...,  1.6406, -0.1959,  1.9292],
         [ 0.3189, -0.5961,  0.2682,  ...,  2.2790,  1.0394,  0.7594],
         [-0.7679, -1.8431, -1.0832,  ..., -0.8743, -0.7678,  1.4047],
         [-0.4181,  0.9909,  1.1471,  ...,  3.3205, -0.6662,  1.1601]],

        [[ 0.4102,  1.7633, -1.3242,  ..., -1.2994,  0.9364,  0.8076],
         [ 0.6146,  0.7651,  1.4400,  ...,  1.1840, -1.6419,  1.2019],
         [-1.0452,  0.6564,  2.5494,  ...,  0.2345,  0.0106,  1.6040],
         [ 1.3965,  0.9862,  0.6768,  ...,  0.7724,  0.7445,  1.0895],
         [ 0.1931, -0.9023, -0.9829,  ..., -2.6426, -0.6831, -0.5114]]])


In [3]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embed size must be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, queries, mask):
        N = queries.shape[0]  # Batch size
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Split embeddings into multiple heads
        values = values.view(N, value_len, self.heads, self.head_dim).transpose(1, 2)
        keys = keys.view(N, key_len, self.heads, self.head_dim).transpose(1, 2)
        queries = queries.view(N, query_len, self.heads, self.head_dim).transpose(1, 2)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])  # Dot product
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.embed_size)
        return self.fc_out(out)

# Example
self_attention = SelfAttention(embed_size, heads=8)
sample_values = torch.randn(2, 5, embed_size)
sample_keys = torch.randn(2, 5, embed_size)
sample_queries = torch.randn(2, 5, embed_size)
print(self_attention(sample_values, sample_keys, sample_queries, mask=None))

tensor([[[ 0.1059, -0.0789, -0.0839,  ...,  0.0024, -0.1556, -0.0331],
         [ 0.1566,  0.0035,  0.2162,  ..., -0.0073, -0.0870, -0.1550],
         [-0.1654, -0.1182, -0.1048,  ..., -0.1170,  0.1022, -0.0191],
         [-0.0157, -0.1965,  0.0076,  ...,  0.0167,  0.2880, -0.0268],
         [ 0.1081, -0.0457, -0.0659,  ..., -0.1559,  0.1625,  0.0477]],

        [[-0.1260, -0.1159,  0.1154,  ...,  0.1198, -0.0679, -0.0425],
         [ 0.0836,  0.0618,  0.0325,  ...,  0.0746,  0.0162,  0.2051],
         [-0.1516, -0.0899, -0.0713,  ..., -0.0303,  0.1963, -0.2460],
         [-0.0095, -0.0186, -0.1020,  ...,  0.1006,  0.1419,  0.1009],
         [-0.0441, -0.0857, -0.0935,  ..., -0.0475, -0.2263,  0.0462]]],
       grad_fn=<ViewBackward0>)


In [4]:
class FeedForward(nn.Module):
    def __init__(self, embed_size, hidden_dim):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(embed_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, embed_size)

    def forward(self, x):
        return self.fc2(torch.relu(self.fc1(x)))

# Example
ffn = FeedForward(embed_size, hidden_dim=2048)
sample_input = torch.randn(2, 5, embed_size)
print(ffn(sample_input))

tensor([[[-0.0909, -0.0775,  0.2253,  ..., -0.1302,  0.2462, -0.0282],
         [-0.1782,  0.2677, -0.1887,  ...,  0.3616,  0.2507, -0.0006],
         [-0.0454,  0.0972, -0.1569,  ..., -0.1528,  0.0155, -0.0808],
         [-0.0219,  0.2562,  0.1462,  ..., -0.2576, -0.2194, -0.0416],
         [-0.1500,  0.0352, -0.1363,  ..., -0.2965, -0.1151,  0.2191]],

        [[ 0.0511, -0.0665, -0.4183,  ..., -0.3174,  0.1728, -0.0676],
         [-0.0664,  0.0125, -0.3269,  ...,  0.0490,  0.1823, -0.0665],
         [ 0.1108,  0.1001, -0.3161,  ..., -0.1170,  0.2251,  0.3858],
         [ 0.1167, -0.0234, -0.0910,  ..., -0.1805, -0.4400,  0.2842],
         [-0.1594, -0.3190,  0.4321,  ..., -0.0313, -0.1045, -0.0744]]],
       grad_fn=<ViewBackward0>)


In [5]:
class LayerNorm(nn.Module):
    def __init__(self, embed_size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(embed_size))
        self.beta = nn.Parameter(torch.zeros(embed_size))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

# Example
layer_norm = LayerNorm(embed_size)
sample_input = torch.randn(2, 5, embed_size)
print(layer_norm(sample_input))

tensor([[[-0.7798, -0.4480,  0.3375,  ..., -1.2159,  0.1344,  0.7867],
         [ 2.4824, -1.5472,  1.7402,  ..., -1.0139,  0.8173,  0.6917],
         [ 0.6812,  1.1265,  0.7304,  ..., -2.9619, -1.7302,  1.7276],
         [-2.1454,  1.0654, -1.6935,  ...,  0.4924, -1.3586,  1.3163],
         [-0.6639,  0.0433, -0.6654,  ..., -0.7002,  2.1036,  0.2055]],

        [[-0.4311, -0.1038,  0.8401,  ...,  2.1543,  0.1376,  2.1675],
         [ 1.0140, -1.7120, -1.3482,  ..., -0.5795, -0.2704,  0.8534],
         [ 0.5510,  1.5075,  0.4955,  ...,  0.1171, -0.4603, -0.8767],
         [-1.2902,  0.7211,  1.8034,  ...,  0.3972, -1.2247, -0.1073],
         [-0.2922, -0.8726, -0.1291,  ...,  0.2879, -0.5819, -1.8562]]],
       grad_fn=<AddBackward0>)


In [6]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, hidden_dim, dropout):
        super(TransformerBlock, self).__init__()
        self.self_attention = SelfAttention(embed_size, heads)
        self.norm1 = LayerNorm(embed_size)
        self.norm2 = LayerNorm(embed_size)
        self.feed_forward = FeedForward(embed_size, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.self_attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

# Example
transformer_block = TransformerBlock(embed_size, heads=8, hidden_dim=2048, dropout=0.1)
sample_values = torch.randn(2, 5, embed_size)
print(transformer_block(sample_values, sample_values, sample_values, mask=None))

tensor([[[ 1.6394, -0.6111,  1.5175,  ...,  1.4980,  0.0967, -0.0000],
         [ 1.2167, -0.9869,  1.1479,  ...,  1.1403, -0.2714, -1.9681],
         [-0.0000, -0.0605,  1.3621,  ...,  0.7021, -0.5099, -0.1085],
         [-0.5430,  0.6556, -0.1436,  ...,  0.9372, -3.2201, -0.4075],
         [-0.9965,  0.1960,  0.6733,  ..., -1.4329,  2.5286, -0.0000]],

        [[-0.9070,  0.0000,  1.2874,  ..., -1.1919,  1.0519,  0.1776],
         [ 1.8984, -0.2736,  1.9871,  ...,  0.9834, -0.7329,  1.5909],
         [ 1.1258, -0.7412,  0.4485,  ..., -0.9039, -0.0667,  1.6592],
         [-0.0563, -0.0043,  0.1151,  ..., -1.9745, -0.9192,  1.5005],
         [ 0.3191,  0.0000,  0.0000,  ...,  0.4360, -1.1291,  0.2762]]],
       grad_fn=<MulBackward0>)


## Playing around with the Torch.nn module

In [7]:
x = torch.randn(3, 4)
print(x)

tensor([[-0.5871, -0.3315, -1.3263, -0.4321],
        [ 1.2011, -0.2415, -1.6131,  0.9473],
        [ 1.0589,  1.8781, -0.2231, -1.3671]])


In [8]:
layer = nn.Linear(in_features=4, out_features=2)
x = torch.randn(3, 4)  # batch size 3, input dim 4
output = layer(x)
print("Linear Output:\n", output)

Linear Output:
 tensor([[ 0.2270,  1.0504],
        [-0.3581, -0.4107],
        [ 0.3391,  1.4584]], grad_fn=<AddmmBackward0>)


In [9]:
relu = nn.ReLU()
sigmoid = nn.Sigmoid()

x = torch.tensor([-1.0, 0.0, 1.0])
print("ReLU:", relu(x))
print("Sigmoid:", sigmoid(x))

ReLU: tensor([0., 0., 1.])
Sigmoid: tensor([0.2689, 0.5000, 0.7311])


In [10]:
criterion = nn.MSELoss()
pred = torch.tensor([0.8, 0.4], requires_grad=True)
target = torch.tensor([1.0, 0.0])
loss = criterion(pred, target)
print("MSE Loss:", loss.item())

MSE Loss: 0.10000000149011612


In [11]:
model = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 2)
)

x = torch.randn(1, 4)
output = model(x)
print("Sequential Model Output:", output)

Sequential Model Output: tensor([[ 0.0334, -0.0974]], grad_fn=<AddmmBackward0>)


In [12]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(4, 8)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(8, 2)

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

net = MyNet()
x = torch.randn(2, 4)
print("Custom Net Output:\n", net(x))

Custom Net Output:
 tensor([[-0.1899, -0.0042],
        [ 0.1392,  0.1560]], grad_fn=<AddmmBackward0>)


In [13]:
dropout = nn.Dropout(p=0.5)
x = torch.ones(10)
print("Before Dropout:", x)
print("After Dropout:", dropout(x))


Before Dropout: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
After Dropout: tensor([0., 2., 0., 0., 2., 2., 2., 2., 2., 2.])
